In [10]:
import pandas as pd
import mlxtend as mle
import pickle
import numpy as np
from lightgbm import LGBMRegressor
from mlxtend.plotting import plot_sequential_feature_selection as plot_sfs
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
%matplotlib inline

In [30]:
res_RF = pd.read_pickle('../data/final/RF_OOF.pck')
res_LGBM = pd.read_pickle('../data/final/LGBM_OOF.pck')
res_RF_id = res_RF.iloc[:,2]
res_LGBM_id = res_LGBM.iloc[:,2]
res_RF.drop(columns =['EPAssetsId'],inplace=True)
res_LGBM.drop(columns =['EPAssetsId'],inplace=True)

res_RF.columns = [f'RF_{x}' for x in res_RF.columns]
res_LGBM.columns = [f'LGBM_{x}' for x in res_LGBM.columns]
res_RF['id'] =res_RF_id
res_LGBM['id'] =res_LGBM_id

df_all = pd.merge(res_LGBM,res_RF,left_on='id',right_on='id').dropna()


In [31]:
cols_y = df_all.columns[df_all.columns.str.contains('gt')]
targets = ['Oil_norm','Gas_norm','Water_norm']
for tgt in targets:
    df_all[tgt] = (df_all[f'LGBM_gt_{tgt}'] +  df_all[f'RF_gt_{tgt}']) * 0.5
df_all.drop(columns=cols_y,inplace=True)
df_all

,LGBM_Oil_norm,LGBM_Gas_norm,LGBM_Water_norm,id,RF_Oil_norm,RF_Gas_norm,RF_Water_norm,Oil_norm,Gas_norm,Water_norm
0,19.194726,8.303552,3.401271,1148297.0,20.176624,6.604340,2.760774,18.589154,11.292888,4.096058
1,12.823057,10.878921,0.510419,2575823.0,15.124304,10.400445,0.648462,10.582239,14.660873,0.612906
2,2.801795,1.945896,0.912790,1097156.0,2.409762,1.826007,0.878624,2.856967,1.474719,0.806492
3,11.509546,2.422085,8.337479,2605869.0,11.816908,2.346907,10.032921,14.941070,0.509449,35.515705
4,28.763746,1.153442,0.576024,1162737.0,27.471949,1.097863,0.522026,22.879497,4.172378,0.341746
...,...,...,...,...,...,...,...,...,...,...
5739,27.715412,2.936809,0.736038,1165844.0,24.428963,2.531521,0.670070,27.211073,1.267408,0.821168
5740,11.634062,1.336480,2.739936,1149123.0,10.736684,0.899263,2.384784,9.987504,0.695178,2.000855
5741,4.433038,1.275862,9.420879,1066824.0,2.665770,1.342667,9.272031,2.714398,1.525730,4.396570
5742,22.927919,8.696605,2.617503,1166708.0,24.214688,7.656352,2.782769,19.798194,6.674497,4.526559


In [44]:
for tgt in targets:
    model = LGBMRegressor(num_leaves =32,n_estimators=1000,objective='mae')
    X = df_all.drop(columns=targets+["id"])
    y = df_all[tgt]
    print(X.columns)
    models = {}
    #model.fit(X,y)
    #models[tgt]=model
    sc = cross_val_score(model,X,y,cv=7,scoring='neg_mean_absolute_error')
    print(np.mean(sc))

Index(['LGBM_Oil_norm', 'LGBM_Gas_norm', 'LGBM_Water_norm', 'RF_Oil_norm',
       'RF_Gas_norm', 'RF_Water_norm'],
      dtype='object')
-2.650350551672967
Index(['LGBM_Oil_norm', 'LGBM_Gas_norm', 'LGBM_Water_norm', 'RF_Oil_norm',
       'RF_Gas_norm', 'RF_Water_norm'],
      dtype='object')
-2.2380465302700334
Index(['LGBM_Oil_norm', 'LGBM_Gas_norm', 'LGBM_Water_norm', 'RF_Oil_norm',
       'RF_Gas_norm', 'RF_Water_norm'],
      dtype='object')
-2.112898510151587


In [ ]:
cross_val_score()